## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
import os
import numpy as np

# Setting path
sys.path.append("..")

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv(os.path.join("..","Weather_Database","WeatherPy_database.csv"))
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,47.26,84,20,8.05,few clouds
1,1,Luderitz,NaN,-26.6481,15.1594,64.18,47,0,4.72,clear sky
2,2,Kavaratti,IN,10.5669,72.6420,81.12,77,100,20.67,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,75.16,94,53,5.01,broken clouds
4,4,Vysokogornyy,RU,50.1167,139.1500,55.65,92,100,2.82,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,47.26,84,20,8.05,few clouds
1,1,Luderitz,NaN,-26.6481,15.1594,64.18,47,0,4.72,clear sky
2,2,Kavaratti,IN,10.5669,72.6420,81.12,77,100,20.67,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,75.16,94,53,5.01,broken clouds
4,4,Vysokogornyy,RU,50.1167,139.1500,55.65,92,100,2.82,overcast clouds
5,5,Souillac,MU,-20.5167,57.5167,69.28,69,60,13.85,broken clouds
6,6,Husavik,IS,66.0449,-17.3389,49.98,82,87,3.51,overcast clouds
8,8,Thompson,CA,55.7435,-97.8558,60.30,71,100,7.76,overcast clouds
9,9,Luebo,CD,-5.3500,21.4167,77.04,44,61,1.79,broken clouds
10,10,Busselton,AU,-33.6500,115.3333,58.48,79,71,9.19,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

601


City_ID                601
City                   601
Country                597
Lat                    601
Lng                    601
Max Temp               601
Humidity               601
Cloudiness             601
Wind Speed             601
Current Description    601
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()
print(len(clean_df))
clean_df.count()

597


City_ID                597
City                   597
Country                597
Lat                    597
Lng                    597
Max Temp               597
Humidity               597
Cloudiness             597
Wind Speed             597
Current Description    597
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,47.26,few clouds,-42.8794,147.3294,
2,Kavaratti,IN,81.12,overcast clouds,10.5669,72.6420,
3,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
4,Vysokogornyy,RU,55.65,overcast clouds,50.1167,139.1500,
5,Souillac,MU,69.28,broken clouds,-20.5167,57.5167,
6,Husavik,IS,49.98,overcast clouds,66.0449,-17.3389,
8,Thompson,CA,60.30,overcast clouds,55.7435,-97.8558,
9,Luebo,CD,77.04,broken clouds,-5.3500,21.4167,
10,Busselton,AU,58.48,broken clouds,-33.6500,115.3333,
11,Vaini,TO,73.96,scattered clouds,-21.2000,-175.2000,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Hobart,AU,47.26,few clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
2,Kavaratti,IN,81.12,overcast clouds,10.5669,72.6420,Green leaf
3,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
5,Souillac,MU,69.28,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
6,Husavik,IS,49.98,overcast clouds,66.0449,-17.3389,Bjarnabúð
8,Thompson,CA,60.30,overcast clouds,55.7435,-97.8558,Burntwood Hotel
10,Busselton,AU,58.48,broken clouds,-33.6500,115.3333,Observatory Guest House
11,Vaini,TO,73.96,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Kapenguria,KE,55.06,light rain,1.2511,35.0895,EDENS HOTEL
14,Jalu,LY,76.98,clear sky,29.0331,21.5482,صاحبة الفخامة


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and <dd>{Max Temp} °F</dd>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))